<a href="https://colab.research.google.com/github/putribuana/EpidermAI-SkinDetection/blob/main/EpidermAI_Transfer_Learning_VGG19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **EpidermAI with Transfer Learning**
Try to use transfer learning from VGG19 (keras)

# Import Module

In [ ]:
!pip install -q kaggle
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import shutil
import random
import splitfolders
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import Model
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.optimizers import RMSprop, Adam, SGD
from keras.callbacks import ReduceLROnPlateau
from google.colab import files
from glob import glob

# **Load Dataset**

In [ ]:
# Upload kaggle.json
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alamardiansyah","key":"4de1498ef16b5da5966b72d443d83511"}'}

In [ ]:
# make dir & copy
!mkdir ~/.kaggle
!cp ./kaggle.json ~/.kaggle

# Change permission of the file
!chmod 600 ~/.kaggle/kaggle.json

# Check the file if exist on directory
!kaggle datasets list

ref                                                       title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
arnabchaki/data-science-salaries-2023                     Data Science Salaries 2023 💸                         25KB  2023-04-13 09:55:16          20814        579  1.0              
fatihb/coffee-quality-data-cqi                            Coffee Quality Data (CQI May-2023)                   22KB  2023-05-12 13:06:39           1959         49  1.0              
darshanprabhu09/stock-prices-for                          Stock prices of Amazon , Microsoft , Google, Apple   85KB  2023-05-16 15:17:16            873         27  1.0              
ashpalsingh1525/imdb-movies-dataset                       IMDB movies dataset             

In [ ]:
# Download datasets
!kaggle datasets download -d riyaelizashaju/isic-skin-disease-image-dataset-labelled

100% 9.09G/9.10G [02:59<00:00, 27.2MB/s]
100% 9.10G/9.10G [02:59<00:00, 54.4MB/s]


In [ ]:
# Unzip
!7za x "/content/isic-skin-disease-image-dataset-labelled.zip"


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 9766946647 bytes (9315 MiB)

Extracting archive: /content/isic-skin-disease-image-dataset-labelled.zip
 16% 4096 Open              --
Path = /content/isic-skin-disease-image-dataset-labelled.zip
Type = zip
Physical Size = 9766946647
64-bit = +

  0%      0% 42 - ISIC_Labelled/Actinic keratosis/ISIC_0027334.jpg                                                            0% 130 - ISIC_Labelled/Actinic keratosis/ISIC_0053478.jpg                                          

# **Split Dataset**

In [ ]:
# Split data to train, validation, & test with ratio (80:10:10)
splitfolders.ratio("/content/ISIC_Labelled", output="split_dataset",seed=1337, ratio=(.7, .2, .1))

Copying files: 20223 files [01:41, 173.59 files/s]

# **Configuration**

In [ ]:
IMAGE_SIZE = [224, 224, 3]
BATCH_SIZE = 64
EPOCHS = 15

TRAIN_DIR = '/content/split_dataset/train'
VAL_DIR = '/content/split_dataset/val'
TEST_DIR = '/content/split_dataset/test'

# **Model Load (VGG19) Using Transfer Learning**

In [ ]:
# Adding preprocessing layer
vgg = VGG19(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)

80134624/80134624 [==============================] - 1s 0us/step


In [ ]:
# Freezing
for layer in vgg.layers:
  layer.trainable = False

# **Add Dense Layers for classifier**

In [ ]:
# Flatten the output layer to 1 dimension
x = Flatten()(vgg.output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.5
x = Dropout(0.5)(x)
# Add a fully connected layer with 256 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = Dense(8, activation='softmax')(x)

# Append the dense network to the base model
model = Model(vgg.input, x)

# Print the model summary. See your dense network connected at the end.
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
# Set the training parameters
model.compile(optimizer = SGD(),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

# **Preprocessing Dataset**

In [ ]:
def train_val_generators(train_dir, val_dir):
  """
  Creates the training and validation data generators
  """
  # Instantiate the ImageDataGenerator class for training
  # Normalize pixel value
  train_datagen = ImageDataGenerator(rescale=1./255.,
                                     rotation_range=20,
                                     horizontal_flip=True)

  # Instantiate the ImageDataGenerator class for validation
  validation_datagen = ImageDataGenerator(rescale=1/255)

  # Pass in the appropriate arguments to the flow_from_directory method
  train_generator = train_datagen.flow_from_directory(TRAIN_DIR,
                                                      batch_size=BATCH_SIZE,
                                                      target_size=(224, 224))

  # Pass in the appropriate arguments to the flow_from_directory method
  validation_generator = validation_datagen.flow_from_directory(VAL_DIR,
                                                                batch_size=BATCH_SIZE,
                                                                target_size=(224, 224))
  return train_generator, validation_generator

In [ ]:
train_generator, validation_generator = train_val_generators(TRAIN_DIR, VAL_DIR)

Found 17728 images belonging to 8 classes.
Found 5062 images belonging to 8 classes.


# **Adjust Learning Rate**

In [ ]:
lr = ReduceLROnPlateau(monitor='val_accuracy',
                       patience=2,
                       verbose=1,
                       factor=0.3,
                       min_lr=0.01)

# **Train The Model**

In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator),
    callbacks=[lr]
)

Epoch 1/15
110/277 [==========>...................] - ETA: 2:08:23 - loss: 1.6353 - accuracy: 0.4615

KeyboardInterrupt: ignored

Look the training history:

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

# **Model Prediction**

# **Model Saved**